## Code to find intersections of ICESat-2 and ATM data

**by Allison Chartrand**

**June 2019 ICESat-2 Hackweek**


In [2]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
from scipy.spatial import distance

In [10]:
%matplotlib widget

**Import Data**

Get ATL06 data and put into dataframe

In [57]:
ATL06filename = '~/xtrak/data_prod/ZachISatData_wSmoooth.csv'
ATMfilename = '~/xtrak/data_prod/ATMprof_2014_slopes_wSmooth.csv'

OutputFilename = '~/xtrak/data_prod/InterX_ATM2014_AllSmooth.csv'

In [38]:
ATL06data = pd.read_csv(ATL06filename,parse_dates=[4])
ATL06data.head()

,lon,lat,h,track,date,x,y,hSm,hSupSm
0,-22.520585,78.700068,579.33765,gt3r,2019-01-04 12:24:25,469494.782922,-1.134613e+06,NaN,NaN
1,-22.520755,78.700244,579.03015,gt3r,2019-01-04 12:24:25,469484.057930,-1.134597e+06,NaN,NaN
2,-22.520925,78.700420,578.78815,gt3r,2019-01-04 12:24:25,469473.331404,-1.134581e+06,NaN,NaN
3,-22.521094,78.700596,578.57214,gt3r,2019-01-04 12:24:25,469462.614457,-1.134564e+06,NaN,NaN
4,-22.521264,78.700773,578.33830,gt3r,2019-01-04 12:24:25,469451.884907,-1.134548e+06,NaN,NaN


Get target points from ATM csv and put into dataframe

In [39]:
ATMdata = pd.read_csv(ATMfilename)

tpoints = ATMdata[['PS_x','PS_y']].copy()
tpoints = tpoints.values
ATMdata.head()

,Unnamed: 0,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along,slope_NS,slope_EW,ATM_elev_Sm,ATM_elev_SupSm
0,0,79.202572,341.714609,527314.046845,-1.047782e+06,32.5185,0.000000,-0.045212,-0.034635,NaN,NaN
1,1,79.202270,341.714622,527329.117517,-1.047811e+06,33.1235,32.996374,0.002895,-0.049338,NaN,NaN
2,2,79.201968,341.714638,527344.243074,-1.047840e+06,32.7155,65.993317,0.017838,-0.074724,NaN,NaN
3,3,79.201666,341.714654,527359.368654,-1.047870e+06,32.1009,98.990275,0.016879,-0.095033,NaN,NaN
4,4,79.201364,341.714670,527374.494258,-1.047899e+06,30.7951,131.987250,0.058634,-0.063234,NaN,NaN


Define nearest neighbor algorithm

In [6]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    closest_dist = np.min(distance.cdist([node], nodes,'euclidean'))
    return closest_index, closest_dist

Make an empty dataframe for intersections
Get query points from ATL06 csv and put into dataframe
Loop through unique dates and ground tracks and find the closest point between each ground track and the flowline
Store intersection along-flow distance, z_ATM, z_ATL06 and other helpful data in Intersections dataframe

In [41]:
Intersections = {'dist_along':[],'ATM_elev':[],'idx_ATM':[],'z_ATL06':[],'t_ATL06':[],'idx_ATL06':[],'gt_ATL06':[]}
Intersections = pd.DataFrame(data = Intersections)
i = 0
for day in ATL06data.date.unique():

    for tr in ATL06data.track.unique():
        close_idx = []
        min_dist = []
        AddDatarow = []

        #dfTran is a single radar transect
        dfTran = ATL06data.query('date == @day & track == @tr')
        if dfTran.shape[0] == 0:
            continue
            
        
        qpoints = dfTran[['x','y']].copy()
        qpoints = qpoints.values

        
        for j in range(len(tpoints)):
            close_idxT,min_distT = closest_node(tpoints[j,:], qpoints)
            close_idx = np.append(close_idx,[close_idxT])
            min_dist = np.append(min_dist,[min_distT])
        if min(min_dist) > 1000:
            continue
        
        tpt_idx = np.argmin(min_dist)
        qpt_idx = int(close_idx[tpt_idx])
        dfTran_idx = dfTran.loc[dfTran['date'] == day].index[0]
        qpt_idx = qpt_idx+dfTran_idx
        
        AddDatarow = {'dist_along':[ATMdata.loc[tpt_idx,'dist_along']],
                      'ATM_elev':[ATMdata.loc[tpt_idx,'ATM_elev_SupSm']],
                      'idx_ATM':[tpt_idx],
                      'z_ATL06':[dfTran.loc[qpt_idx,'hSupSm']],
                      't_ATL06':[dfTran.loc[qpt_idx,'date']],
                      'idx_ATL06':[qpt_idx],
                      'gt_ATL06':[dfTran.loc[qpt_idx,'track']]}
        AddDatarow = pd.DataFrame(data = AddDatarow)
        Intersections = Intersections.append(AddDatarow,ignore_index=True)
#         Intersections.reset_index(drop=True)
        

In [42]:
Intersections.head()


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,46456.889444,38.33215,1359.0,35.501406,2019-02-15 10:09:55,1576.0,gt1l
1,46391.934750,38.17740,1357.0,34.084645,2019-02-15 10:09:55,4931.0,gt1r
2,75537.773997,90.38570,2205.0,69.272560,2019-01-16 01:09:49,39474.0,gt3r
3,69049.697427,27.24080,2006.0,28.789461,2019-01-16 01:09:49,19999.0,gt1l
4,69114.630647,27.18605,2008.0,29.224519,2019-01-16 01:09:49,23921.0,gt1r


Plot 

In [43]:
f, ax = plt.subplots()
plt.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],c=Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'], c = Time,s=1)
plt.plot(Intersections['dist_along']/1000,Intersections['ATM_elev'],'.')
# cb = f.colorbar()
plt.colorbar()

# cb.ax.set_yticklabels(df.index.strftime('%b %Y'))

FigureCanvasNbAgg()

In [97]:
plt.close('all')

Sort the data by date and track

In [52]:
IntersectionsSort = Intersections.sort_values(by=['t_ATL06', 'gt_ATL06'])
IntersectionsSort.reset_index()

,index,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,161,49165.513987,44.89925,1442.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l
1,162,49099.625709,44.74530,1440.0,42.156155,2018-10-18 15:53:52,621074.0,gt1r
2,163,45936.554970,36.59180,1343.0,34.448452,2018-10-18 15:53:52,624012.0,gt2l
3,164,45871.380221,35.97035,1341.0,32.827094,2018-10-18 15:53:52,626687.0,gt2r
4,52,91172.740382,294.13865,2671.0,NaN,2018-10-21 05:21:45,205271.0,gt2r
5,51,94353.390910,270.69060,2769.0,255.108995,2018-10-21 05:21:45,205803.0,gt3r
6,103,104090.781799,415.62490,3034.0,361.948000,2018-10-25 05:13:27,398282.0,gt1l
7,104,104059.444579,414.06135,3033.0,391.111190,2018-10-25 05:13:27,400497.0,gt1r
8,105,107139.323794,425.31270,3133.0,NaN,2018-10-25 05:13:27,401953.0,gt2l
9,106,107262.251679,425.31270,3137.0,410.575870,2018-10-25 05:13:27,403762.0,gt2r


Compute slope between adjacent intersections of beam pairs

In [59]:
InterX = IntersectionsSort.values
InterX = InterX

Dist = np.diff(IntersectionsSort['dist_along'].values)
Dist[Dist == 0] = float('NaN')
InterXslopeATL06 = np.diff(IntersectionsSort['z_ATL06'].values)/Dist
InterXslopeATM = np.diff(IntersectionsSort['ATM_elev'].values)/Dist

# i = 0
# for i in Dist():
InterXslopeATL06[np.abs(Dist) > 150] = float('NaN')
InterXslopeATL06 = np.append(InterXslopeATL06,[0])

InterXslopeATM[np.abs(Dist) > 150] = float('NaN')
InterXslopeATM = np.append(InterXslopeATM,[float('NaN')])


/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()
/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


In [49]:
IntersectionsSort['dist_along'].values

array([ 49165.51398724,  49099.62570875,  45936.55496996,  45871.38022056,
        91172.7403818 ,  94353.39090985, 104090.78179945, 104059.44457892,
       107139.32379431, 107262.25167929, 110366.99918223, 110431.00023435,
        82369.41961212,  82267.22006931,  78966.05719298,  78898.16197799,
        75570.53164979,  75505.07556135,  49792.65570448,  49891.83295597,
        53093.19030555,  53158.26916432,  56385.61812834,  56451.49329452,
        66150.65550138,  66745.34497847,  69601.43646664,  69666.28014006,
        72786.57024526,  72852.08152054,  43829.59800902,  43829.59800902,
        82505.62653136,  82573.74935764,  85715.09928844,  85783.14836676,
        88882.21852224,  88948.16141327,  60000.44749032,  59934.139129  ,
        56780.20616169,  56714.51376458,  53582.32386413,  53484.30453456,
        98609.87669249,  98676.19030358, 101964.39830549, 101964.39830549,
       104998.92026576, 105092.18744702,  76718.73681071,  76619.99420482,
        73345.10181576,  

Add 90m slope arrays back into IntersectionsSort and save to csv

In [60]:
IntersectionsSort = IntersectionsSort.assign(slope_ATM= InterXslopeATM)
IntersectionsSort = IntersectionsSort.assign(slope_ATL06 = InterXslopeATL06)
IntersectionsSort.head()

,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,slope_ATM,slope_ATL06
161,49165.513987,44.89925,1442.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l,0.002337,0.009302
162,49099.625709,44.74530,1440.0,42.156155,2018-10-18 15:53:52,621074.0,gt1r,NaN,NaN
163,45936.554970,36.59180,1343.0,34.448452,2018-10-18 15:53:52,624012.0,gt2l,0.009535,0.024877
164,45871.380221,35.97035,1341.0,32.827094,2018-10-18 15:53:52,626687.0,gt2r,NaN,NaN
52,91172.740382,294.13865,2671.0,NaN,2018-10-21 05:21:45,205271.0,gt2r,NaN,NaN


Save to file

In [61]:

IntersectionsSort.to_csv(OutputFilename,index=False)